In [545]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [546]:
import numpy as np
from numpy import array
import numpy.polynomial.polynomial as poly
import pandas as pd
import datetime as dt
import seaborn
import matplotlib.pylab as plt
import os
import random
import shutil
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [547]:
seaborn.set(rc={'figure.figsize':(16,8.27)})

In [548]:
df = pd.read_csv('/kaggle/input/stock-exchange-data/indexData.csv')

In [549]:
df.head()

In [550]:
df.info

In [551]:
df.describe()

In [552]:
df.isnull().sum()

In [553]:
df["Date"] = pd.to_datetime(df["Date"])

In [554]:
df_NYA = df[df["Index"].isin(['NYA'])].sort_values('Date')

In [555]:
df_NYA

In [556]:
plt.plot(df_NYA["Date"],df_NYA["Close"],label='Row Data')
plt.legend()

In [557]:
df_NYA_data_close = df_NYA[["Date","Close"]]

In [558]:
df_NYA_data_close

In [559]:
df_NYA_data_close.isnull().sum()

In [560]:
df_NYA_data_close.index

In [561]:
np.where(df_NYA_data_close['Close'].isnull())[0]

In [562]:
df_NYA_data_close.iloc[[289]]

In [563]:
df_NYA_data_close_withoutnan = df_NYA_data_close.dropna()

In [564]:
df_NYA_data_close_withoutnan.isnull().sum()

In [565]:
np.where(df_NYA_data_close_withoutnan['Close'].isnull())[0]

In [566]:
data_train = df_NYA_data_close_withoutnan[df_NYA_data_close_withoutnan.Date < "2010-01-01"]
data_test = df_NYA_data_close_withoutnan[df_NYA_data_close_withoutnan.Date >= "2010-01-01"]

In [567]:
data_train

In [568]:
data_train.Date = data_train.Date.map(dt.datetime.toordinal)

In [569]:
X_train = data_train.Date[:, np.newaxis]

In [570]:
X_train

In [571]:
y_train = data_train.Close

In [572]:
y_train.isnull().sum()

In [573]:
tree = DecisionTreeRegressor(criterion='mse',max_depth=50).fit(X_train, y_train)

In [574]:
linear_reg = LinearRegression().fit(X_train, y_train)

In [575]:
model_rf = RandomForestRegressor(n_estimators=100).fit(X_train, y_train)

In [576]:
X_all = df_NYA_data_close_withoutnan.Date.map(dt.datetime.toordinal)[:, np.newaxis]

In [577]:
X_all

In [578]:
np.where(df_NYA_data_close_withoutnan['Date'] == '1966-01-05')[0]

In [579]:
X_all[3]

In [580]:
pred_tree = tree.predict(X_all)
pred_model_rf = model_rf.predict(X_all)
pred_lr = linear_reg.predict(X_all)
X = np.array(data_train.Date)
y = np.array(data_train.Close)
model_coef_polyfit = poly.polyfit(X, y, 3)
X_new = np.array(df_NYA_data_close_withoutnan.Date.map(dt.datetime.toordinal))
ffit = poly.polyval(X_new, model_coef_polyfit)
X_new = pd.DataFrame(X_new,columns=['Date'])
X_new = X_new.Date.map(dt.datetime.fromordinal)

In [581]:
plt.plot(df_NYA_data_close_withoutnan.Date,df_NYA_data_close_withoutnan.Close,label='Row Data')
plt.plot(df_NYA_data_close_withoutnan.Date,pred_tree,label='Decision Tree')
plt.plot(df_NYA_data_close_withoutnan.Date,pred_lr,label='Linear Regression')
plt.plot(df_NYA_data_close_withoutnan.Date,pred_model_rf,label='Random Forest')
plt.plot(X_new,ffit,label='Fit using polyfit')
plt.legend()

In [582]:
tree.score(X_all,df_NYA_data_close_withoutnan.Close)

In [583]:
linear_reg.score(X_all,df_NYA_data_close_withoutnan.Close)

In [584]:
model_rf.score(X_all,df_NYA_data_close_withoutnan.Close)

In [585]:
df_N100 = df[df["Index"].isin(['N100'])].sort_values('Date')

In [586]:
new_index = np.arange(0,len(df_N100.Close),1)

In [587]:
len(df_N100.Close)

In [588]:
df_N100_data_close = df_N100[["Date","Close"]]
df_N100_data_close

In [589]:
df_N100_data_close.index

In [590]:
df_N100_data_close.isnull().sum()

In [591]:
df_N100_data_close_withoutnan = df_N100_data_close.dropna()

In [592]:
len(df_N100_data_close_withoutnan.Close)

In [593]:
df_N100_data_close_withoutnan.isnull().sum()

In [594]:
data_train_N100 = df_N100_data_close_withoutnan[df_N100_data_close_withoutnan.Date < "2015-01-01"]
data_test_N100 = df_N100_data_close_withoutnan[df_N100_data_close_withoutnan.Date >= "2015-01-01"]

In [595]:
df_N100_data_close_withoutnan.index

In [596]:
data_train_N100.Date = data_train_N100.Date.map(dt.datetime.toordinal)

In [597]:
data_train_N100.Close

In [598]:
X_train_N100 = data_train_N100.Date[:, np.newaxis]
y_train_N100 = data_train_N100.Close

In [599]:
tree_N100 = DecisionTreeRegressor(criterion='mse',max_depth=50).fit(X_train_N100, y_train_N100)
linear_reg_N100 = LinearRegression().fit(X_train_N100, y_train_N100)
model_rf_N100 = RandomForestRegressor(n_estimators=10).fit(X_train_N100, y_train_N100)

In [600]:
X_all_N100 = df_N100_data_close_withoutnan.Date.map(dt.datetime.toordinal)[:, np.newaxis]

In [601]:
pred_tree_N100 = tree_N100.predict(X_all_N100)
pred_lr_N100 = linear_reg_N100.predict(X_all_N100)
pred_model_rf_N100 = model_rf_N100.predict(X_all_N100)

In [602]:
X_N100 = np.array(data_train_N100.Date)
y_N100 = np.array(data_train_N100.Close)
model_coef_polyfit = poly.polyfit(X_N100, y_N100, 2)
X_new_N100 = np.array(df_N100_data_close_withoutnan.Date.map(dt.datetime.toordinal))
ffit_N100 = poly.polyval(X_new_N100, model_coef_polyfit)
X_new_N100 = pd.DataFrame(X_new_N100,columns=['Date'])
X_new_N100 = X_new_N100.Date.map(dt.datetime.fromordinal)

In [603]:
plt.plot(df_N100_data_close_withoutnan.Date,df_N100_data_close_withoutnan.Close,label='Row Data')
plt.plot(df_N100_data_close_withoutnan.Date,pred_tree_N100,label='Decision Tree')
plt.plot(df_N100_data_close_withoutnan.Date,pred_lr_N100,label='Linear Regression')
plt.plot(df_N100_data_close_withoutnan.Date,pred_model_rf_N100,label='Random Forest')
plt.plot(X_new_N100,ffit_N100,label='Fit using Polyfit')
plt.legend()

In [604]:
# split data into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [605]:
data_train = df_NYA_data_close_withoutnan[df_NYA_data_close_withoutnan.Date < "2010-01-01"]
data_test = df_NYA_data_close_withoutnan[df_NYA_data_close_withoutnan.Date >= "2010-01-01"]
data_train

In [606]:
# choose a number of time steps
n_steps_in, n_steps_out = 10, 1
# split data into samples
X, y = split_sequence(data_train.Close, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X.shape)

In [607]:
# model
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(LSTM(200, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['accuracy'])
model.summary()

In [608]:
# fit model
history = model.fit(X, y, epochs=10, verbose=2, validation_split=0.33)

In [609]:
plt.plot(history.history['loss'],label='Train')
plt.plot(history.history['val_loss'],label='Validation')
plt.legend()

In [611]:
# demonstrate prediction
# choose a number of time steps
n_steps_in, n_steps_out = 10, 1
# split data into samples
X, y = split_sequence(data_test.Close, n_steps_in, n_steps_out)
print(X.shape)
x_input = X.reshape((X.shape[0], X.shape[1], n_features))
yhat_1 = model.predict(x_input, verbose=0)
print(yhat_1)

In [612]:
test_mean_1 = np.mean(yhat_1,axis=1)
print(test_mean_1)

In [613]:
plt.plot(df_NYA_data_close_withoutnan.Date,df_NYA_data_close_withoutnan.Close,'k',label='Row Data')
plt.plot(data_test.Date[10:],test_mean_1,label='RNN using LSTM')
plt.legend()

In [614]:
plt.plot(data_test.Date[10:],data_test.Close[10:],'-b',label='Test Data')
plt.plot(data_test.Date[10:],test_mean_1,'r',label='RNN using LSTM')
plt.legend()

In [615]:
r2_score(data_test.Close[10:], test_mean_1)

In [616]:
mean_squared_error(data_test.Close[10:],test_mean_1)

In [617]:
((test_mean_1-data_test.Close[10:])**2).sum()/len(test_mean_1)

In [618]:
((test_mean_1-data_test.Close[10:])**2).mean()

In [619]:
# choose a number of time steps
n_steps_in, n_steps_out = 10, 9
# split into samples
X, y = split_sequence(data_train.Close, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X.shape)

In [620]:
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(LSTM(200, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['accuracy'])
model.summary()

In [621]:
# fit model
history = model.fit(X, y, epochs=10, verbose=2, validation_split=0.33)

In [622]:
plt.plot(history.history['loss'],label='Train')
plt.plot(history.history['val_loss'],label='Validation')
plt.legend()

In [623]:
plt.plot(history.history['accuracy'],label='Train')
plt.plot(history.history['val_accuracy'],label='Validation')
plt.legend()

In [624]:
# demonstrate prediction
# choose a number of time steps
n_steps_in, n_steps_out = 10, 9
# split data into samples
X, y = split_sequence(data_test.Close, n_steps_in, n_steps_out)
print(X.shape)
x_input = X.reshape((X.shape[0], X.shape[1], n_features))
yhat_2 = model.predict(x_input, verbose=0)
print(yhat_2)

In [625]:
test_mean_2 = np.mean(yhat_2,axis=1)

In [626]:
plt.plot(data_test.Date[18:],data_test.Close[18:],'-b',label='Test Data')
plt.plot(data_test.Date[18:],test_mean_2,'r',label='RNN using LSTM')
plt.legend()

In [627]:
r2_score(data_test.Close[18:], test_mean_2)

In [628]:
mean_squared_error(data_test.Close[18:],test_mean_2)

In [629]:
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split data into samples
X, y = split_sequence(data_train.Close, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X.shape)

In [630]:
# model
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(LSTM(200, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['accuracy'])
model.summary()

In [631]:
# fit model
history = model.fit(X, y, epochs=10, verbose=2, validation_split=0.33)

In [632]:
plt.plot(history.history['loss'],label='Train')
plt.plot(history.history['val_loss'],label='Validation')
plt.legend()
#plt.ylim(0,10000)

In [633]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [634]:
# demonstrate prediction
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split data into samples
X, y = split_sequence(data_test.Close, n_steps_in, n_steps_out)
print(X.shape)
x_input = X.reshape((X.shape[0], X.shape[1], n_features))
yhat_3 = model.predict(x_input, verbose=0)
print(yhat_3)

In [635]:
test_mean_3 = np.mean(yhat_3,axis=1)

In [636]:
plt.plot(df_NYA_data_close_withoutnan.Date,df_NYA_data_close_withoutnan.Close,'k',label='Row Data')
plt.plot(data_test.Date[4:],test_mean_3,label='RNN using LSTM')
plt.legend()

In [637]:
plt.plot(data_test.Date[4:],data_test.Close[4:],'-b',label='Test Data')
plt.plot(data_test.Date[4:],test_mean_3,'r',label='RNN using LSTM')
plt.legend()

In [638]:
r2_score(data_test.Close[4:], test_mean_3)

In [639]:
mean_squared_error(data_test.Close[4:],test_mean_3)

In [640]:
data_train_N100 = df_N100_data_close_withoutnan[df_N100_data_close_withoutnan.Date < "2015-01-01"]
data_test_N100 = df_N100_data_close_withoutnan[df_N100_data_close_withoutnan.Date >= "2015-01-01"]

In [641]:
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(data_train_N100.Close, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X.shape)

In [642]:
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(LSTM(200, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['accuracy'])
model.summary()

In [643]:
# fit model
history = model.fit(X, y, epochs=10, verbose=2, validation_split=0.33)

In [644]:
plt.plot(history.history['loss'],label='Train')
plt.plot(history.history['val_loss'],label='Validation')
plt.legend()
plt.ylim(0,1000)

In [645]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [646]:
# demonstrate prediction
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(data_test_N100.Close, n_steps_in, n_steps_out)
print(X.shape)
x_input = X.reshape((X.shape[0], X.shape[1], n_features))
yhat_N100 = model.predict(x_input, verbose=0)
print(yhat_N100)

In [647]:
test_mean_N100 = np.mean(yhat_N100,axis=1)
test_mean_N100

In [648]:
plt.plot(df_N100_data_close_withoutnan.Date,df_N100_data_close_withoutnan.Close,'k',label='Row Data')
plt.plot(data_test_N100.Date[4:],test_mean_N100,label='RNN using LSTM')
plt.legend()

In [649]:
plt.plot(data_test_N100.Date[4:],data_test_N100.Close[4:],'-b',label='Test Data')
plt.plot(data_test_N100.Date[4:],test_mean_N100,'r',label='RNN using LSTM')
plt.legend()

In [650]:
r2_score(data_test_N100.Close[4:], test_mean_N100)

In [651]:
mean_squared_error(data_test_N100.Close[4:],test_mean_N100)

In [652]:
# choose a number of time steps
n_steps_in, n_steps_out = 10, 1
# split into samples
X, y = split_sequence(data_train_N100.Close, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X.shape)

In [653]:
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(LSTM(200, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['accuracy'])
model.summary()

In [654]:
# fit model
history = model.fit(X, y, epochs=10, verbose=2, validation_split=0.33)

In [655]:
plt.plot(history.history['loss'],label='Train')
plt.plot(history.history['val_loss'],label='Validation')
plt.legend()
plt.ylim(0,1000)

In [657]:
# demonstrate prediction
# choose a number of time steps
n_steps_in, n_steps_out = 10, 1
# split into samples
X, y = split_sequence(data_test_N100.Close, n_steps_in, n_steps_out)
print(X.shape)
x_input = X.reshape((X.shape[0], X.shape[1], n_features))
yhat_N100_3 = model.predict(x_input, verbose=0)
print(yhat_N100_3)

In [658]:
test_mean_N100_3 = np.mean(yhat_N100_3,axis=1)
test_mean_N100_3

In [659]:
plt.plot(df_N100_data_close_withoutnan.Date,df_N100_data_close_withoutnan.Close,'k',label='Row Data')
plt.plot(data_test_N100.Date[10:],test_mean_N100_3,label='RNN using LSTM')
plt.legend()

In [660]:
plt.plot(data_test_N100.Date[10:],data_test_N100.Close[10:],'-b',label='Test Data')
plt.plot(data_test_N100.Date[10:],test_mean_N100_3,'r',label='RNN using LSTM')
plt.legend()

In [661]:
r2_score(data_test_N100.Close[10:], test_mean_N100_3)

In [662]:
mean_squared_error(data_test_N100.Close[10:],test_mean_N100_3)